<a href="https://colab.research.google.com/github/HalgasAdrian/CS5230-Coursework/blob/main/HW2_KMEANS_and_Gaussian_Mixtures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HW2 KMNEANS and Gaussian Mixtures


Problem 1: KMeans Theory

Problem 2: KMeans on data

Using Euclidian distance or dot product similarity (choose one per dataset, you can try other similarity metrics).

A) run KMeans on the MNIST Dataset, try K=10

B) run KMeans on the FASHION Dataset, try K=10

C) run KMeans on the 20NG Dataset, try K=20

You can use a library for distance/similarity but you have to implement your own kmeans (EM steps, termination criteria etc).
For all three datasets, evaluate the KMeans objective for a higher K (for example double) or smaller K(for example half).
For all three datasets, evaluate external clustering performance using data labels and performance metrics Purity and Gini Index (see [A] book section 6.9.2).

D) run soft Kmeans for MNIST dataset, use K=10. You can try beta=0.1, beta=1, beta=10. Evaluate performance.

A) Run KMeans on the MNIST Dataset, try K = 10

In [1]:
import os

base_url = 'https://web.archive.org/web/20220331230320/http://yann.lecun.com/exdb/mnist/'
files = [
    'train-images-idx3-ubyte.gz',
    'train-labels-idx1-ubyte.gz',
    't10k-images-idx3-ubyte.gz',
    't10k-labels-idx1-ubyte.gz'
]

# Directory to store MNIST data
os.makedirs('/content/mnist', exist_ok=True)

# Download each file
for file in files:
    url = f"{base_url}{file}"
    output_path = f"/content/mnist/{file}"
    !wget -O {output_path} {url}

print("All MNIST files downloaded.")

--2025-02-03 02:22:34--  https://web.archive.org/web/20220331230320/http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Resolving web.archive.org (web.archive.org)... 207.241.237.3
Connecting to web.archive.org (web.archive.org)|207.241.237.3|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://web.archive.org/web/20220331225332/http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz [following]
--2025-02-03 02:22:35--  https://web.archive.org/web/20220331225332/http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Reusing existing connection to web.archive.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 9912422 (9.5M) [application/x-gzip]
Saving to: ‘/content/mnist/train-images-idx3-ubyte.gz’

/content/mnist/trai 100%[===================>]   9.45M  14.1MB/s    in 0.7s    

2025-02-03 02:22:37 (14.1 MB/s) - ‘/content/mnist/train-images-idx3-ubyte.gz’ saved [9912422/9912422]

--2025-02-03 02:22:37--  https://web.archive

In [2]:
import gzip
import numpy as np

# Function to extract gzip files
def extract_gz(file_path, out_path):
    with gzip.open(file_path, 'rb') as f_in:
        with open(out_path, 'wb') as f_out:
            f_out.write(f_in.read())

# Extract MNIST files
mnist_dir = '/content/mnist'
for file in files:
    gz_path = os.path.join(mnist_dir, file)
    out_path = gz_path.replace('.gz', '')
    extract_gz(gz_path, out_path)

print("All MNIST files extracted.")

# Function to parse MNIST image files
def load_mnist_images(file_path):
    with open(file_path, 'rb') as f:
        f.read(16)  # Skip header
        data = np.frombuffer(f.read(), dtype=np.uint8)
        return data.reshape(-1, 28, 28)

# Function to parse MNIST label files
def load_mnist_labels(file_path):
    with open(file_path, 'rb') as f:
        f.read(8)  # Skip header
        data = np.frombuffer(f.read(), dtype=np.uint8)
        return data

# Load training and test data
X_train_mnist = load_mnist_images('/content/mnist/train-images-idx3-ubyte')
y_train_mnist = load_mnist_labels('/content/mnist/train-labels-idx1-ubyte')
X_test_mnist = load_mnist_images('/content/mnist/t10k-images-idx3-ubyte')
y_test_mnist = load_mnist_labels('/content/mnist/t10k-labels-idx1-ubyte')

print("MNIST data successfully loaded.")

All MNIST files extracted.
MNIST data successfully loaded.


KMeans from scratch

In [3]:
import numpy as np

class KMeans:
    def __init__(self, n_clusters=10, max_iter=300, tol=1e-4):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.tol = tol
        self.centroids = None
        self.labels = None

    def fit(self, X):
        n_samples, n_features = X.shape

        # Initialize centroids randomly
        random_indices = np.random.choice(n_samples, self.n_clusters, replace=False)
        self.centroids = X[random_indices]

        for i in range(self.max_iter):
            # Assign clusters
            distances = self._compute_distances(X)
            self.labels = np.argmin(distances, axis=1)

            # Update centroids
            new_centroids = np.array([X[self.labels == k].mean(axis=0) for k in range(self.n_clusters)])

            # Check for convergence
            if np.linalg.norm(new_centroids - self.centroids) < self.tol:
                break

            self.centroids = new_centroids

    def _compute_distances(self, X):
        distances = np.zeros((X.shape[0], self.n_clusters))
        for k in range(self.n_clusters):
            distances[:, k] = np.linalg.norm(X - self.centroids[k], axis=1)
        return distances

    def predict(self, X):
        distances = self._compute_distances(X)
        return np.argmin(distances, axis=1)

Kmeans on MNIST Dataset with K = 10

In [4]:
# Flatten the MNIST images
X_train_mnist_flat = X_train_mnist.reshape(X_train_mnist.shape[0], -1)

# Normalize the data
X_train_mnist_flat = X_train_mnist_flat / 255.0

# Run KMeans
kmeans_mnist = KMeans(n_clusters=10)
kmeans_mnist.fit(X_train_mnist_flat)

# Evaluate the KMeans objective
def kmeans_objective(X, centroids, labels):
    return np.sum([np.linalg.norm(X[labels == k] - centroids[k])**2 for k in range(centroids.shape[0])])

objective_mnist = kmeans_objective(X_train_mnist_flat, kmeans_mnist.centroids, kmeans_mnist.labels)
print(f"KMeans Objective for MNIST (K=10): {objective_mnist}")

KMeans Objective for MNIST (K=10): 2352952.706814831


In [5]:
from sklearn.metrics import confusion_matrix

def purity_score(y_true, y_pred):
    confusion_mat = confusion_matrix(y_true, y_pred)
    return np.sum(np.amax(confusion_mat, axis=0)) / np.sum(confusion_mat)

def gini_index(y_true, y_pred):
    confusion_mat = confusion_matrix(y_true, y_pred)
    cluster_sizes = np.sum(confusion_mat, axis=0)
    cluster_purities = np.amax(confusion_mat, axis=0) / cluster_sizes
    return 1 - np.sum(cluster_purities**2)

purity_mnist = purity_score(y_train_mnist, kmeans_mnist.labels)
gini_mnist = gini_index(y_train_mnist, kmeans_mnist.labels)

print(f"Purity for MNIST (K=10): {purity_mnist}")
print(f"Gini Index for MNIST (K=10): {gini_mnist}")

Purity for MNIST (K=10): 0.5828
Gini Index for MNIST (K=10): -3.375299139898414
